In [1]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from VAN_new import *
import pandas as pd


    
def get_column(matrix, i):
    return torch.tensor([matrix[j][i] for j in range(len(matrix))])

def energie1D(spin): 
    spin_copie=spin.clone()
    spin_copie[spin_copie==0]=-1
    spin_copie_1 = torch.roll(spin_copie, -1)
    spin_copie_2 = torch.roll(spin_copie, 1)    
    energie=- torch.sum(spin_copie_1*spin_copie+spin_copie_2*spin_copie)
    return energie

def energie2D(lattice):
    energie = 0 
    for i in range(len(lattice)):
        energie+=energie1D(lattice[0])
    for j in range(len(lattice[0])):
        column = get_column(lattice, j)
        energie+=energie1D(column)
    return energie


def log_prob_energie(beta, energie):
    return -beta*energie


def log_prob_target_energie(spins, beta):
    
    log_probs = torch.ones(spins.shape[0]) * np.log(0.001)
    for i in range(len(log_probs)):
        racine=spins[i].shape[0]
        racine=(int(np.sqrt(racine)))
        lattice = spins[i].reshape(racine, racine)
        log_probs[i] = log_prob_energie(beta, energie2D(lattice))
    return log_probs 



In [2]:

### WITH ANNEALING
for taille in  [    144, 169, 14*14, 15*15, 16*16, 17*17, 18*18, 19*19, 20*20 ]: 

    mymodel1 = VAN(taille)
    racine=int(np.sqrt(taille))

    betas=[0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1]

    for beta in betas: 
        print( taille, beta)
        magnetisations_list=[]
        losses = train(mymodel1, lambda x:  log_prob_target_energie(x, beta), batch_size=200, n_iter=1000, lr=0.001)
        mysample=mymodel1.sample(1000)
        magnetisations=[]
        for spin in mysample:
            
            magnetisations.append(torch.mean(spin))
        plt.hist(magnetisations, bins=20, edgecolor='black') 
        plt.xlabel('Magnetisation of the spin')
        plt.ylabel('Number of spins')
        plt.title('Magnetisation of the spins for beta =' + str(beta) + ' on a ' + str(racine) + 'x' + str(racine) + ' lattice')
        plt.savefig('./figures/magnetisation_test_annealing for beta= '+str(beta)+ ', racine=' +str(racine) + '.png')
        magnetisations_list.append(magnetisations)
        plt.show()
        pd.DataFrame(magnetisations_list).to_csv('./magnetisations/magnetisations_test_annealing for beta= '+str(beta)+ ' and racine = ' + str(racine) + '.csv')


144 0.005
Epoch 0: 43.459476470947266
Epoch 100: 40.995872497558594
Epoch 200: 37.22169494628906


ValueError: Expected parameter probs (Tensor of shape (200,)) of distribution Bernoulli(probs: torch.Size([200])) to satisfy the constraint Interval(lower_bound=0.0, upper_bound=1.0), but found invalid values:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan,

In [ ]:


### WITHOUT ANNEALING


betas=[ 0.6, 0.7, 0.8, 0.9, 1, 1.1]

for beta in betas: 
    magnetisations_list=[]

    for i in range(10):
        my_model = VAN(taille)
        losses = train(my_model, lambda x:  log_prob_target_energie(x, beta), batch_size=200, n_iter=1000, lr=0.01)
        mysample=my_model.sample(1000)
        magnetisations=[]
        for spin in mysample:
            magnetisations.append(torch.mean(spin))
        plt.hist(magnetisations, bins=20, edgecolor='black') 
        plt.xlabel('Magnetisation of the spin')
        plt.ylabel('Number of spins')
        plt.title('Magnetisation of the spins for beta =' + str(beta))
        plt.savefig('./figures/magnetisation_for beta= '+str(beta)+ 'test n° ' +str(i) + '.png')
        magnetisations_list.append(magnetisations)
        plt.show()
    pd.DataFrame(magnetisations_list).to_csv('./magnetisations/magnetisations for beta= '+str(beta)+'.csv')
